In [1]:
%matplotlib inline
%matplotlib notebook
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme


from config import CONFIG
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math
import numpy as np
import pandas as pd
import sys
import statistics
import time

import scipy.optimize as spo
import cvxopt as opt
from cvxopt import blas, solvers

In [2]:
# uncomment and execute line to try a new theme
# set_nb_theme('onedork')
# set_nb_theme('chesterish')
# set_nb_theme('grade3')
# set_nb_theme('oceans16')
# set_nb_theme('solarizedl')
# set_nb_theme('solarizedd')
# set_nb_theme('monokai')

In [3]:
path = CONFIG['PATH']
pairs = CONFIG['PAIRS']
start_date='2017-01-01'
end_date='2017-12-31'
dates=pd.date_range(start_date, end_date)
merged_candlesticks=pd.DataFrame(index=dates)

for pair in pairs:
    candlesticks = pd.read_csv(path + "/data/"+ pair.replace('/','-') +"/candlesticks.csv", parse_dates=True, usecols=['date','close'], na_values=['nan'])
    candlesticks['date'] = pd.Series([dt.datetime.fromtimestamp(int(ts)).strftime('%Y-%m-%d')
                                     for ts in candlesticks['date']]).values
    candlesticks = candlesticks.set_index('date')
    candlesticks = candlesticks.rename(columns={'close':pair})
    merged_candlesticks=merged_candlesticks.join(candlesticks)
    merged_candlesticks.dropna()
print(merged_candlesticks.head(5)) 

             ETH/USDT  LTC/USDT     BTC/USDT  XMR/USDT
2017-01-01   8.389086  4.605872  1019.000001    16.020
2017-01-02   9.653000  4.590000  1037.100000    16.000
2017-01-03  11.100000  4.603085  1136.000000    18.200
2017-01-04  10.090000  4.241776   998.800000    15.999
2017-01-05  10.080000  3.844500   896.000000    13.989


In [4]:
def plot_data(df, title="Prices"):
    ax = df.plot(title=title)
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    plt.show()

def normalize_data(df):
    return df / df.iloc[0,:]

def compute_daily_returns(df):
    daily_returns = df.copy()
    daily_returns[1:] = (df[1:] / df[:-1].values) - 1
    daily_returns.iloc[0,:] = 0
    return daily_returns

def scatter_plot(df, base_pair):
    for pair in pairs:
        if pair != base_pair:
            df.plot(kind='scatter', x=base_pair, y=pair)
            beta, alpha = np.polyfit(daily_returns[base_pair], daily_returns[pair],1)
            plt.title("Beta and Alpha for " + pair + " in relation to " + base_pair)
            plt.plot(df[base_pair], beta * df[base_pair] + alpha, '-', color='r')
            print('alpha',alpha)
            print('beta',beta)

In [5]:
#print(merged_candlesticks.loc['2017-01-01':'2017-12-31', ['BTS/BTC','STEEM/BTC']].head(5))

# Daily Returns

In [6]:
daily_returns=compute_daily_returns(merged_candlesticks)
# plot_data(daily_returns)

# Plot Distribution of Daily Returns

In [7]:
# for pair in pairs:
#     fig, ax = plt.subplots()
#     daily_returns[pair].hist(bins=20, label=pair)
#     mean = daily_returns[pair].mean()
#     std = daily_returns[pair].std()
#     ax.set_xlabel("Frequency of returns")
#     ax.set_ylabel("Return Amount")
#     ax.set_title(pair + " Daily Returns")
#     plt.axvline(mean,color='w',linestyle='dashed', linewidth=1)
#     plt.axvline(std,color='r',linestyle='dashed', linewidth=1)
#     plt.axvline(-std,color='r',linestyle='dashed', linewidth=1)
# plt.show()    

# Daily Return Beta & Alpha

* Alpha - pos = return is higher than asset it is compared to
* Beta - How reactive the market is

In [8]:
# scatter_plot(daily_returns, 'BTC/USDT')

# Correlation Coef

In [9]:
print(daily_returns.corr(method='pearson'))

          ETH/USDT  LTC/USDT  BTC/USDT  XMR/USDT
ETH/USDT  1.000000  0.347535  0.367542  0.486397
LTC/USDT  0.347535  1.000000  0.382820  0.356871
BTC/USDT  0.367542  0.382820  1.000000  0.437815
XMR/USDT  0.486397  0.356871  0.437815  1.000000


# Normalize data

In [10]:
normalized=normalize_data(merged_candlesticks)
# plot_data(normalized, "Nomalized prices")

# Daily Portfolio Values

* fetch prices
* normalize data
* multiply nomalized data by allocation amounts
* multiply allocated amounts by your initail portfolio start val (using 10 in this example)
* sum all columns


In [11]:
coins = []
allocs = []
start_val = 100000

for pair in pairs:
    allocs.append(pairs[pair])
    coins.append(pair)

allocated = normalized * allocs
pos_vals = allocated * start_val
port_val = pos_vals.sum(axis=1)

In [12]:
daily_rf = 0

daily_returns = daily_returns[1:]
cum_ret=(port_val[-1]/port_val[0])-1
avg_daily_ret = daily_returns.mean()
std_daily_ret = daily_returns.std()
sharpe_ratio = np.mean(daily_returns - daily_rf)/np.std(daily_returns)
sharpe_ratio_annualized = math.sqrt(365)*sharpe_ratio
sharpe_ratio_annualized

ETH/USDT    3.982082
LTC/USDT    3.022562
BTC/USDT    3.162978
XMR/USDT    2.720902
dtype: float64

# Portfolio Optimization

In [13]:
np.random.seed(1)
 
# length of artificial time series
n_obs = 1000
 
# number of different assets
n_assets = 4
 
# sample Nx4 data series matrix
artificial_returns = np.random.randn(n_obs, n_assets) + 0.05
artificial_returns

array([[ 1.67434536, -0.56175641, -0.47817175, -1.02296862],
       [ 0.91540763, -2.2515387 ,  1.79481176, -0.7112069 ],
       [ 0.3690391 , -0.19937038,  1.51210794, -2.01014071],
       ..., 
       [ 0.12341633,  0.46602616, -1.82920004,  0.62545885],
       [ 0.15206241,  1.23430372, -0.74484305, -0.07590305],
       [-0.9103464 , -0.79391327,  0.67834172,  0.58721449]])

In [35]:
def rand_weights(n):
    ''' Produces n random weights that sum to 1 '''
    k = np.random.rand(n)
    return k / sum(k)


def random_portfolio(returns):
    ''' 
    Returns the mean and standard deviation of returns for a random portfolio
    '''

    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    
    # This recursion reduces outliers to keep plots pretty
    if sigma > 2:
        return random_portfolio(returns)
    return mu, sigma

n_portfolios = 500
means, stds = np.column_stack([
    random_portfolio(daily_returns) 
    for _ in range(n_portfolios)
])

In [44]:
def optimal_portfolio(returns):
    n = len(returns)
    returns = np.ascontiguousarray(returns, dtype=np.float32)
    
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns)) 
    pbar = np.matrix(returns.mean())
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    return np.asarray(wt), returns, risks

In [45]:
weights, returns, risks = optimal_portfolio(daily_returns)

plt.plot(stds, means, 'o')
plt.ylabel('mean')
plt.xlabel('std')
plt.plot(risks, returns, 'y-o')

TypeError: 'q' must be a 'd' matrix with one column